In [110]:
import numpy as np
import robustsp as rsp

def repeated_median_filter(x):
    N = len(x)
    
    xFRM = np.zeros(N) # filter cleaned signal
    
    # Works for signals of prime length, therefore, signal is split into two
    # overlapping segments which are of prime length
    x_split = rsp.split_into_prime(x)
    
    # for each prime segment do repeated median filtering
    for ii in range(len(x_split[0,:])):
        x_part = x_split[:,ii]
        
        # Fourier frequencies in descending order
        wr = rsp.order_wk(x_part)[0]

        # Length of the prime time segment
        N_prime = len(x_part)
        
        # time vector
        t = range(N_prime)
        
        # number of fourier coefficients
        K =(N_prime-1)/2

        # Repeated median estimate of 
        # cosine coefficients at w(k)
        ARM = np.zeros(int(K))
        
        # Repeated median estimate of sine
        # coefficients at w(k)
        BRM = np.zeros(int(K))
        
        # repeated median transform: estimate ARM and BRM starting with stongest
        # w(k), subtract from time series, repeat M times
        
        # number of iterations, as recommended in the paper by Tatum and Hurvich
        M = 2
        
        # remove a robust location estimate 
        # (the sample median)
        xm = np.median(x_part)
        xt = x_part - xm

        Auv = np.zeros((N_prime, N_prime))
        Buv = np.zeros((N_prime, N_prime))
        
        A = np.zeros(int(K))
        B = np.zeros(int(K))
        
        for m in range(M):
            for k in range(int(K)):
                for u in range(N_prime):
                    for v in range(N_prime):
                        if u != v:
                            Auv[u,v] =\
                            (xt[u]*np.sin(wr[k]*v)\
                            -xt[v]*np.sin(wr[k]*u))\
                            /np.sin(wr[k]*(v-u))
                            
                            Buv[u,v] =\
                            (xt[v]*np.cos(wr[k]*u)\
                            -xt[u]*np.cos(wr[k]*v))\
                            /np.sin(wr[k]*(v-u))
                A[k] = np.median(np.median(Auv,axis=0))

                B[k] = np.median(np.median(Buv,axis=0))
                
                xt -=  (A[k]*np.cos(wr[k]*t) + B[k]*np.sin(wr[k]*t))
                
                ARM[k] +=  A[k]
                
                BRM[k] +=  B[k]

        # recover the core process by regression of the repeated median estimates
        # onto the independent parameters

        if ii == 0:
            xFRM1 = np.array(xm)
            for k in range(int(K)):
                sumAB = ARM[k]*np.cos(wr[k]*t)+\
                BRM[k]*np.sin(wr[k]*t)

                xFRM1 = xFRM1 + sumAB
        elif ii == 1:
            xFRM2 = np.array(xm)
            for k in range(int(K)):
                sumAB = ARM[k]*np.cos(wr[k]*t)+\
                BRM[k]*np.sin(wr[k]*t)

                xFRM2 = xFRM2 + sumAB    
    if ii==0:
        xFRM=np.array(xFRM1)
    elif ii==1:
        xFRM[:N-N_prime] = xFRM1[:N-N_prime]
        xFRM[N-N_prime:N_prime] =\
        (xFRM1[N-N_prime:N_prime]+ xFRM2[N-N_prime:N_prime])/2
        xFRM[N_prime:] = xFRM2[::N-N_prime]
    return xFRM, ARM, BRM

In [111]:
'''
ans =

    0.8633
    2.1831
    4.4311
    3.9249
    4.2850
    6.7307
    7.0526
    6.3869
    9.0304
   10.1188
   
   b =

   -1.7242   -0.7530   -0.6595


c =

   -1.3105   -0.0876   -1.4737
'''

repeated_median_filter([1,2,3,4,5,6,7,8,9,10])

(array([ 0.86326317,  2.18309314,  4.43105324,  5.4864588 ,  4.8868801 ,
         7.53042681,  8.61882474,  3.86326317,  6.9864588 , 10.11882474]),
 array([-1.72424986, -0.7530204 , -0.65946657]),
 array([-1.31051709, -0.08758488, -1.47372481]))

In [109]:
x = [1,2,3,4,5,6,7,8,9,10]

N = len(x)

xFRM = np.zeros(N) # filter cleaned signal

# Works for signals of prime length, therefore, signal is split into two
# overlapping segments which are of prime length
x_split = rsp.split_into_prime(x)

# for each prime segment do repeated median filtering
ii=0
x_part = x_split[:,ii]

# Fourier frequencies in descending order
wr = rsp.order_wk(x_part)[0]

# Length of the prime time segment
N_prime = len(x_part)

# time vector
t = np.arange(N_prime) # ok 

# number of fourier coefficients
K =(N_prime-1)/2

# Repeated median estimate of 
# cosine coefficients at w(k)
ARM = np.zeros(int(K))

# Repeated median estimate of sine
# coefficients at w(k)
BRM = np.zeros(int(K))

# repeated median transform: estimate ARM and BRM starting with stongest
# w(k), subtract from time series, repeat M times

# number of iterations, as recommended in the paper by Tatum and Hurvich
M = 2

# remove a robust location estimate 
# (the sample median)
xm = np.median(x_part)
xt = x_part - xm
# stimmt print(xt)
Auv = np.zeros((N_prime, N_prime))
Buv = np.zeros((N_prime, N_prime))

A = np.zeros(int(K))
B = np.zeros(int(K))
M=1

# ok print(xt)
# ok print(wr)

k = 0
print('k: %i'% k)
for u in range(N_prime):
    for v in range(N_prime):
        if u != v:
            Auv[u,v] =\
            (xt[u]*np.sin(wr[k]*v)\
            -xt[v]*np.sin(wr[k]*u))\
            /np.sin(wr[k]*(v-u))

            Buv[u,v] =\
            (xt[v]*np.cos(wr[k]*u)\
            -xt[u]*np.cos(wr[k]*v))\
            /np.sin(wr[k]*(v-u))
A[k] = np.median(np.median(Auv,axis=0)) # ok

B[k] = np.median(np.median(Buv,axis=0)) # ok
print(A[k], wr[k])
xt -=  (A[k]*np.cos(wr[k]*t) + B[k]*np.sin(wr[k]*t)) # false
print(xt)
ARM[k] +=  A[k] # ok

BRM[k] +=  B[k] # ok

k=1
print('k: %i'% k)
for u in range(N_prime):
    for v in range(N_prime):
        if u != v:
            Auv[u,v] =\
            (xt[u]*np.sin(wr[k]*v)\
            -xt[v]*np.sin(wr[k]*u))\
            /np.sin(wr[k]*(v-u))

            Buv[u,v] =\
            (xt[v]*np.cos(wr[k]*u)\
            -xt[u]*np.cos(wr[k]*v))\
            /np.sin(wr[k]*(v-u))
A[k] = np.median(np.median(Auv,axis=0))

B[k] = np.median(np.median(Buv,axis=0))

xt -=  A[k]*np.cos(wr[k]*t) - B[k]*np.sin(wr[k]*t)

ARM[k] +=  A[k]

BRM[k] +=  B[k]


k: 0
-2.2469796037174663 1.7951958020513104
[-0.7530204  -0.9205832  -3.72736527  0.13437493  3.6675628   0.67844793
  0.9205832 ]
k: 1


0
0
0
0
0
0


IndexError: index 6 is out of bounds for axis 1 with size 6

In [2]:
import scipy.io as sio

y = sio.loadmat('y')['y'].flatten()
ARMm = sio.loadmat('rmf')['ARM'].flatten()
xFRMm = sio.loadmat('rmf')['xFRM'].flatten()
BRMm = sio.loadmat('rmf')['BRM'].flatten()

In [3]:
xFRM,ARM,BRM = repeated_median_filter(y)

KeyboardInterrupt: 

In [4]:
int(5.7)

5

In [ ]:
np.array_equal(xFRM,xFRMm), np.array_equal(ARM,ARMm), np.array_equal(BRM,BRMm)

In [4]:
import numpy as np
import robustsp as rsp
import scipy.io as sio

x = sio.loadmat('y')['y'].flatten()
##########
N = len(x)

xFRM = np.zeros(N) # filter cleaned signal

# Works for signals of prime length, therefore, signal is split into two
# overlapping segments which are of prime length
x_split = rsp.split_into_prime(x)

In [ ]:
ii = 0
x_part = np.array(x_split[:,ii])

# Fourier frequencies in descending order

wr = rsp.order_wk(x_part)[0]

In [7]:
wr

array([0.64189712, 0.67892965, 0.69127383, 0.6048646 , 0.59252042,
       0.703618  , 0.71596218, 0.72830635, 0.6542413 , 0.61720877,
       0.76533888, 0.53079954, 0.74065053, 0.66658547, 0.19750681,
       0.48142284, 0.62955295, 0.30860439, 0.54314372, 0.56783207,
       1.97506807, 0.46907867, 2.29601663, 2.32070499, 0.23453933,
       1.99975642, 1.9133472 , 0.79002723, 0.91346898, 2.93791376,
       0.92581316, 2.07382148, 2.64165355, 0.82705976, 0.8023714 ,
       2.30836081, 0.58017625, 1.74052874, 0.0493767 , 0.77768305,
       0.87643646, 1.90100302, 1.86397049, 2.1602307 , 1.78990544,
       1.95037972, 1.19738502, 1.49364523, 2.12319818, 0.83940393,
       2.08616565, 0.40735779, 0.24688351, 2.48117927, 1.71584039,
       0.20985098, 0.5554879 , 1.83928214, 2.03678895, 2.13554235,
       2.40711421, 1.64177534, 0.98753404, 1.55536611, 0.06172088,
       1.27145007, 0.0987534 , 0.43204614, 1.32082677, 0.85174811,
       1.80224962, 0.45673449, 2.91322541, 2.110854  , 0.08640

In [ ]:
# for each prime segment do repeated median filtering
for ii in range(len(x_split[0,:])):
    x_part = x_split[:,ii]

    # Fourier frequencies in descending order
    wr = rsp.order_wk(x_part)[0]

    # Length of the prime time segment
    N_prime = len(x_part)

    # time vector
    t = range(N_prime)

    # number of fourier coefficients
    K =(N_prime-1)/2

    # Repeated median estimate of 
    # cosine coefficients at w(k)
    ARM = np.zeros(int(K))

    # Repeated median estimate of sine
    # coefficients at w(k)
    BRM = np.zeros(int(K))

    # repeated median transform: estimate ARM and BRM starting with stongest
    # w(k), subtract from time series, repeat M times

    # number of iterations, as recommended in the paper by Tatum and Hurvich
    M = 2

    # remove a robust location estimate 
    # (the sample median)
    xm = np.median(x_part)
    xt = x_part - xm

    Auv = np.zeros((N_prime-1, N_prime-1))
    Buv = np.zeros((N_prime-1, N_prime-1))

    for m in range(M):
        for k in range(int(K)):
            for u in range(N_prime):
                for v in range(N_prime):
                    if u != v:
                        Auv[u,v] =\
                        (xt[u]*np.sin(wr[k]*v)\
                        -xt[v]*np.sin(wr[k]*v))\
                        /np.sin(wr[k]*(v-u))

                        Buv[u,v] =\
                        (xt[u]*np.cos(wr[k]*v)\
                        -xt[v]*np.cos(wr[k]*v))\
                        /np.sin(wr[k]*(v-u))
            A[k] = np.median(np.median(Auv,axis=0))

            B[k] = np.median(np.median(Buv,axis=0))

            xt = xt - A[k]*np.cos(wr[k]*t) - B[k]*np.sin(wr[k]*t)

            ARM[k] = ARM[k] + A[k]

            BRM[k] = BRM[k] + B[k]
    # recover the core process by regression of the repeated median estimates
    # onto the independent parameters

    if ii == 0:
        xFRM1 = np.array(xm)
        for k in range(int(K)):
            sumAB = ARM[k]*np.cos(wr[k]*t)+\
            BRM[k]*np.sin(wr[k]*t)

            xFRM1 += sumAB
    elif ii == 1:
        xFRM2 = np.array(xm)
        for k in range(int(K)):
            sumAB = ARM[k]*np.cos(wr[k]*t)+\
            BRM[k]*np.sin(wr[k]*t)

            xFRM2 += sumAB

if ii==0:
    xFRM=xFRM1
elif ii==1:
    xFRM[:N-N_prime] = xFRM1[:N-N_prime]
    xFRM[N-N_prime+1:N_prime] =\
    (xFRM1[N-N_prime+1:N_prime]+ xFRM2[N-N_prime+1:N_prime])/2
    xFRM[N_prime+1:] = xFRM2[::N-N_prime]

In [1]:
import numpy as np
import robustsp as rsp

xxx = np.array([-0.2269  , -0.1625   , 0.6901  ,  0.5558  , -1.1203   ,-1.5327 ,  -1.0979,-1.4158 ,   0.0596,   -0.4113 ,  -0.3680])
# should be 1.1424    0.5712    2.8560    2.2848    1.7136
rsp.order_wk(xxx)[0]

array([0.57119866, 0.        , 2.28479466, 1.71359599, 1.14239733])

In [15]:
np.array([3,1,2,5,4]).argsort()

array([1, 2, 0, 4, 3], dtype=int64)